In [1]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pandapower as pp
from pandapower.networks import case2848rte

In [2]:
random.seed(10)
np.random.seed(20)

In [3]:
# Load any case file you need
net = case2848rte()

# Same settings as in the synthetic data generation
net.sgen.in_service = False
net.gen.max_p_mw = 500
net.line.max_loading_percent = 10000
net.trafo.max_loading_percent = 1e20
net.ext_grid.min_p_mw = 0

In [4]:
# # Change bus name from 'Name' to 'Bus_no'
# net.bus.name.name = 'Bus_no'
# # Bus no. starts from 0
# net_index = net.bus.name.astype(int) - 1

net_index = np.arange(net.bus.shape[0])

#### Get node features from the case network

In [5]:
# Create DataFrame to store network data
network = pd.DataFrame()

# Set network index
network.index = net_index

In [6]:
# Get features from 'bus
network['vn_kv'] = net.bus.vn_kv.values

# Get features from 'load'
net.load.index = net.load.bus
network['load_p_mw'] = net.load.p_mw

# Get features from 'gen'
net.gen.index = net.gen.bus
network['max_gen_p_mw'] = net.gen.max_p_mw
network['min_gen_p_mw'] = net.gen.min_p_mw

# Get features from 'ext_grid' (ext_grid bus no: 68)
net.ext_grid.index = net.ext_grid.bus
network['max_gen_p_mw'][net.ext_grid.index.values] = net.ext_grid.max_p_mw
network['min_gen_p_mw'][net.ext_grid.index.values] = net.ext_grid.min_p_mw

# Get features from 'poly_cost'
# This needs manual work
gen_price = pd.read_csv('./gen_price.csv', index_col=0, header=0)
network['price_1'] = gen_price.price_term1
network['price_2'] = gen_price.price_term2

# Additional features
network['x_pos'] = net.bus_geodata.x
network['y_pos'] = net.bus_geodata.y

#### Get edge index and attributes

In [7]:
branch = net.line[['from_bus', 'to_bus']]

In [8]:
branch_attr = net.line[['c_nf_per_km', 'r_ohm_per_km', 'x_ohm_per_km']]

In [9]:
trafo = net.trafo[['hv_bus', 'lv_bus']]

In [10]:
trafo_attr = net.trafo[['sn_mva', 'tap_neutral', 'tap_pos', 'tap_step_percent', 'vk_percent']]

#### Save node and edge informartion

In [11]:
network.to_csv('network/node_features.csv')
branch.to_csv('network/branch_index.csv')
branch_attr.to_csv('network/branch_attr.csv')
trafo.to_csv('network/trafo_index.csv')
trafo_attr.to_csv('network/trafo_attr.csv')

#### Save normalized data and scaler

In [12]:
from sklearn.preprocessing import MinMaxScaler
import joblib

# Normalize branch atrributes
branch_attr_scaler = MinMaxScaler()
branch_attr_scaled = branch_attr_scaler.fit_transform(branch_attr)
pd.DataFrame(branch_attr_scaled).to_csv('network/branch_attr_scaled.csv')
joblib.dump(branch_attr_scaler, 'network/branch_attr_scaler.scaler')

# Normalize transformer attributes
trafo_attr_scaler = MinMaxScaler()
trafo_attr_scaled = trafo_attr_scaler.fit_transform(trafo_attr)
pd.DataFrame(trafo_attr_scaled).to_csv('network/trafo_attr_scaled.csv')
joblib.dump(trafo_attr_scaler, 'network/trafo_attr_scaler.scaler')

['network/trafo_attr_scaler.scaler']